In [8]:
import cv2
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing import image
from keras.api.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.api.models import Sequential
from keras.api.applications import VGG16
from keras.api.optimizers import Adam
from PIL import ImageFile

train_dir = "/workspace/sdp/redd/train"
test_dir = "/workspace/sdp/redd/test"


In [9]:
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

def apply_gamma_correction(image, gamma=1.2):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)


def preprocess_image(image):
    image = (image * 255).round().astype(np.uint8)

    # Apply CLAHE and Gamma Correction
    image = apply_clahe(image)
    image = apply_gamma_correction(image)

    # Convert back to float32 [0, 1] for the model
    return image.astype(np.float32) / 255

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    preprocessing_function=preprocess_image,  # Use your custom function directly
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image  # Apply preprocessing to test set as well
)


# Load Training & Testing Data
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 15000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [20]:
# Load Pretrained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the base model

# Define Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,879,299 (56.76 MB)

 Trainable params: 164,611 (643.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [21]:
import pickle

epochs = 5
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=epochs,
    verbose=2
)
model.save("my_model-ilu.h5")
with open("history-ilu.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("Model and training history saved successfully!")


Epoch 1/5
469/469 - 576s - 1s/step - accuracy: 0.5897 - loss: 0.8447 - val_accuracy: 0.8033 - val_loss: 0.4807
Epoch 2/5
469/469 - 738s - 2s/step - accuracy: 0.7488 - loss: 0.5627 - val_accuracy: 0.8410 - val_loss: 0.3887
Epoch 3/5
469/469 - 875s - 2s/step - accuracy: 0.7855 - loss: 0.4940 - val_accuracy: 0.8487 - val_loss: 0.3504
Epoch 4/5
469/469 - 795s - 2s/step - accuracy: 0.8020 - loss: 0.4577 - val_accuracy: 0.8650 - val_loss: 0.3204
Epoch 5/5
469/469 - 827s - 2s/step - accuracy: 0.8135 - loss: 0.4358 - val_accuracy: 0.8670 - val_loss: 0.3083


Model and training history saved successfully!
